In [1]:
from models.cli2p import CLI2P
from PIL import Image
import torch

d:\anaconda\envs\pt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image1 = model.img_preprocessor(Image.open("examples/pokemon.jpeg")   ).unsqueeze(0).to(device)   # .shape=torch.Size([1, 3, 224, 224])
image2 = model.img_preprocessor(Image.open("examples/pokemon.jpeg")   ).unsqueeze(0).to(device)   # .shape=torch.Size([1, 3, 224, 224])
torch.cat([image1, image2]).shape

NameError: name 'model' is not defined

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

config_dict ={
    "device": device
}
model = CLI2P(config_dict)
model.cuda()
model.eval()

image1 = model.img_preprocessor(Image.open("examples/pokemon.jpeg")).unsqueeze(0).to(device)   # .shape=torch.Size([1, 3, 224, 224])
image2 = model.img_preprocessor(Image.open("examples/pokemon.jpeg")).unsqueeze(0).to(device)   # .shape=torch.Size([1, 3, 224, 224])
image = torch.cat([image1, image2])
# text = tokenize(["杰尼龟", "妙蛙种子", "小火龙", "皮卡丘"]).to(device)
text = model.text_preprocessor(["杰尼龟",'妙蛙种子']).to(device) # .shape=[1,52]  ===> [ [101(开始标志), 3345('杰'的索引), 2225('尼'的索引), 7991('龟'的索引) ， 0 ，0， 0，... 0  ] ]  ---> 所谓索引就是该字符在 models/vocab.txt 中的行数

with torch.no_grad():
    image_features = model.feat_extrator.encode_image(image)
    text_features = model.feat_extrator.encode_text(text)
    # 对特征进行归一化，请使用归一化后的图文特征用于下游任务
    image_features /= image_features.norm(dim=-1, keepdim=True)  # 
    text_features /= text_features.norm(dim=-1, keepdim=True)    
    print(f'image_features.shape:{image_features.shape}')  # image_features.shape:torch.Size([1, 512])   可见 图像编码 与 文本编码 最后都是编码成为 512 
    print(f'text_features.shape:{text_features.shape}')    # text_features.shape:torch.Size([1, 512])
    mix_f = model(image, text)
    print(f"mix_f.shape:{mix_f.shape}")

Loading vision model config from e:\explore\CLI2P\models\model_configs\ViT-B-16.json
Loading text model config from e:\explore\CLI2P\models\model_configs\RoBERTa-wwm-ext-base-chinese.json
Model info {'embed_dim': 512, 'image_resolution': 224, 'vision_layers': 12, 'vision_width': 768, 'vision_patch_size': 16, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 12, 'text_type_vocab_size': 2}


d:\anaconda\envs\pt\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


image_features.shape:torch.Size([2, 512])
text_features.shape:torch.Size([2, 512])
mix_f.shape:torch.Size([2, 1024])


In [7]:
text = model.text_preprocessor("老人与海 THE OLD MAN AND 美 H319.4:1712.45/ THESEA")
text.shape

torch.Size([1, 52])

In [ ]:
r

In [13]:
image_features

tensor([[ 0.0955, -0.0375,  0.0165,  ..., -0.0066,  0.0392,  0.0115],
        [ 0.0955, -0.0375,  0.0165,  ..., -0.0066,  0.0392,  0.0115]],
       device='cuda:0', dtype=torch.float16)

In [14]:
text_features

tensor([[ 0.1174, -0.0163,  0.0040,  ...,  0.0345,  0.0659, -0.0065],
        [ 0.1010, -0.0084, -0.0572,  ...,  0.0056,  0.0452,  0.0021]],
       device='cuda:0', dtype=torch.float16)